# Shor's algorithm for integer factoring

This example implementation of Shor's algorithm is [adapted from ProjectQ](https://github.com/ProjectQ-Framework/ProjectQ/blob/develop/examples/shor.py).

In [1]:
import math
import random
from fractions import Fraction

try:
    from math import gcd
except ImportError:
    from fractions import gcd

from pyqrack import QrackSimulator, Pauli

import os
os.environ['QRACK_QUNIT_SEPARABILITY_THRESHOLD']='0.001'

`toFactor` is the number we are factoring. (You may change it.) 

In [2]:
toFactor=2**14-1

The entire algorithm is implemented below. For the first variant, to factor an integer of `n` qubits or less, we have a footprint of `2n`, but PyQrack's `pown()` is limited in qubit width to single GPU "pages," (usually 2 qubits less than maximum for the GPU,) and the operation is "fully entangling" of the internal representation. 

In [3]:
%%time
base = int(random.random() * toFactor)
if not gcd(base, toFactor) == 1:
    print("Chose non-relative prime, (without need for quantum computing):")
    print("Factor: {}".format(gcd(base, toFactor)))
else:
    qubitCount = math.ceil(math.log2(toFactor))
    sim = QrackSimulator(2 * qubitCount)

    qi = [i for i in range(qubitCount)]
    qo = [(i + qubitCount) for i in range(qubitCount)]

    # run the quantum subroutine
    for i in qi:
        sim.h(i)
    sim.pown(base, toFactor, qi, qo)
    sim.iqft(qi)
    
    b = [Pauli.PauliZ] * qubitCount
    y = sim.measure_pauli(b, qi)
    r = Fraction(y).limit_denominator(toFactor - 1).denominator

    # try to determine the factors
    if r % 2 != 0:
        r *= 2
    apowrhalf = pow(base, r >> 1, toFactor)
    f1 = gcd(apowrhalf + 1, toFactor)
    f2 = gcd(apowrhalf - 1, toFactor)
    fmul = f1 * f2
    if (not fmul == toFactor) and f1 * f2 > 1 and (toFactor // fmul) * fmul == toFactor:
        f1, f2 = f1 * f2, toFactor // (f1 * f2)
    if f1 * f2 == toFactor and f1 > 1 and f2 > 1:
        print("Factors found : {} * {} = {}".format(f1, f2, toFactor))
    else:
        print("Failed: Found {} and {}".format(f1, f2))

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_Gen9_HD_Graphics_NEO.ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
Factors found : 3 * 5461 = 16383
CPU times: user 8.91 s, sys: 89.3 ms, total: 9 s
Wall time: 11 s


The second implementation requires `2n+2` qubits in footprint to factor a number of `n` qubits or less. However, these methods do not resort to "fully entangled representation," and they are not limited to single GPU "pages."

In [4]:
toFactor=15

In [5]:
def cmul_native(sim, i, a, maxN, qo, qa):
    sim.mcmuln(a, [i], maxN, qo, qa)
    for o in range(len(qa)):
        sim.cswap([i], qa[o], qo[o])
    sim.mcdivn(a, [i], maxN, qo, qa)

def phase_root_n(sim, n, q):
    sim.mtrx([1, 0, 0, -1**(1.0 / (1<<(n - 1)))], q)

In [6]:
%%time
# Based on https://arxiv.org/abs/quant-ph/0205095
base = random.randrange(2, toFactor)
factor = gcd(base, toFactor)
if not factor == 1:
    print("Chose non-relative prime, (without need for quantum computing):")
    print("Factors found : {} * {} = {}".format(factor, toFactor // factor, toFactor))
else:
    qubitCount = math.ceil(math.log2(toFactor))
    maxN = 1<<qubitCount
    sim = QrackSimulator(2 * qubitCount + 2)

    qo = [i for i in range(qubitCount)]
    qa = [(i + qubitCount) for i in range(qubitCount)]
    qi = 2 * qubitCount
    qm = 2 * qubitCount + 1

    m_results = []

    # Run the quantum subroutine.
    # First, set the multiplication output register to identity, 1.
    sim.x(qo[0])
    for i in range(qubitCount):
        sim.h(qi)
        cmul_native(sim, qi, 1 << i, toFactor, qo, qa)

        # We use the single control qubit "trick" referenced in Beauregard:
        for j in range(len(m_results)):
            if m_results[j]:
                phase_root_n(sim, j + 2, qi)

        m_results.append(sim.m(qi))
        if m_results[-1]:
            sim.x(qi)

    y = 0
    for i in range(len(m_results)):
        if m_results[i]:
            y |= 1<<i
    r = Fraction(y).limit_denominator(toFactor - 1).denominator

    # try to determine the factors
    if r % 2 != 0:
        r *= 2
    apowrhalf = pow(base, r >> 1, toFactor)
    f1 = gcd(apowrhalf + 1, toFactor)
    f2 = gcd(apowrhalf - 1, toFactor)
    fmul = f1 * f2
    if (not fmul == toFactor) and f1 * f2 > 1 and (toFactor // fmul) * fmul == toFactor:
        f1, f2 = f1 * f2, toFactor // (f1 * f2)
    if f1 * f2 == toFactor and f1 > 1 and f2 > 1:
        print("Factors found : {} * {} = {}".format(f1, f2, toFactor))
    else:
        print("Failed: Found {} and {}".format(f1, f2))

Chose non-relative prime, (without need for quantum computing):
Factors found : 3 * 5 = 15
CPU times: user 76 µs, sys: 1 µs, total: 77 µs
Wall time: 57.9 µs


Finally, we can't apply exact Shor's integer factoring algorithm to many simulated qubits, but Qrack is capable of a high-qubit-width approximation with a tunable footprint, (always in `2n+2` qubits, but with variable quantities of entanglement and superposition).

Two approximations make this classically efficient to simulate:
1. `hCount` limits entanglement and superposition, by limiting the maximum number of Hadamard gates applied. After `hCount` is exhausted, the algorithm completes in a "semi-classical" probabilistic manner.
2. Modular controlled multiplication is actually only pseudo-modular, here, which further reduces entanglement. Almost without loss of generality, we always "collapse" the last semi-classical qubit into |0> state, which self-consistently avoids the requirement of "fully-modular" controlled multiplication, but it truncates possible factorization outputs, by forcing the highest bit to be |0>. (However, up to the partially excluded limit of perfect squares, there is commonly at least one factor from any pair of factorss in our search space.) 

In [7]:
toFactor=2**60-1
hCount = 2

In [8]:
def cmul(sim, i, a, maxN, qo, qa):
    for o in range(len(qo)):
        partMul = (a * (1 << o)) % maxN
        if partMul == 0:
            continue
        sim.mcadd(partMul, [qo[o]], qa)

def cdiv(sim, i, a, maxN, qo, qa):
    for o in range(len(qo)):
        partMul = (a * (1 << o)) % maxN
        if partMul == 0:
            continue
        sim.mcsub(partMul, [qo[o]], qa)

def cmul_in_place(sim, i, a, maxN, qo, qa):
    cmul(sim, i, a, maxN, qo, qa)
    for o in range(len(qa)):
        sim.cswap([i], qa[o], qo[o])
    cdiv(sim, i, a, maxN, qo, qa)

def phase_root_n(sim, n, q):
    sim.mtrx([1, 0, 0, -1**(1.0 / (1<<(n - 1)))], q)

In [10]:
%%time
# Based on https://arxiv.org/abs/quant-ph/0205095
while True:
    base = random.randrange(2, toFactor)
    factor = gcd(base, toFactor)
    if not factor == 1:
        print("Chose non-relative prime, (without need for quantum computing):")
        print("Factors found : {} * {} = {}".format(factor, toFactor // factor, toFactor))
        break
    else:
        qubitCount = math.ceil(math.log2(toFactor))
        maxN = 1<<qubitCount
        sim = QrackSimulator(2 * qubitCount + 2, isPaged=False, isOpenCL=False)

        qo = [i for i in range(qubitCount)]
        qa = [(i + qubitCount) for i in range(qubitCount)]
        qi = 2 * qubitCount
        qm = 2 * qubitCount + 1

        m_results = []

        # Run the quantum subroutine.
        # First, set the multiplication output register to identity, 1.
        sim.x(qo[0])
        for i in range(qubitCount):
            if i < hCount:
                sim.h(qi)
                # This is a pseudo-modular operation. However, it's efficient to simulate!
                cmul_in_place(sim, qi, 1 << i, toFactor, qo, qa)
            elif i < (qubitCount - 1) and random.getrandbits(1) > 0:
                # Approximate with a semi-classical controlled-multipication.
                for o in range(len(qo)):
                    partMul = ((1 << i) * (1 << o)) % maxN
                    if partMul == 0:
                        continue
                    sim.add(partMul, qo)

            # We use the single control qubit "trick" referenced in Beauregard:
            for j in range(len(m_results)):
                if m_results[j]:
                    phase_root_n(sim, j + 2, qi)

            if i == (qubitCount - 1):
                m_results.append(sim.force_m(qi, False))
            else:
                m_results.append(sim.m(qi))
            if m_results[-1]:
                sim.x(qi)

        y = 0
        for i in range(len(m_results)):
            if m_results[i]:
                y |= 1<<i
        r = Fraction(y).limit_denominator(toFactor - 1).denominator

        # try to determine the factors
        if r % 2 != 0:
            r *= 2
        apowrhalf = pow(base, r >> 1, toFactor)
        f1 = gcd(apowrhalf + 1, toFactor)
        f2 = gcd(apowrhalf - 1, toFactor)
        fmul = f1 * f2
        if (not fmul == toFactor) and f1 * f2 > 1 and (toFactor // fmul) * fmul == toFactor:
            f1, f2 = f1 * f2, toFactor // (f1 * f2)
        if f1 * f2 == toFactor and f1 > 1 and f2 > 1:
            print("Factors found : {} * {} = {}".format(f1, f2, toFactor))
            break
        else:
            print("Failed: Found {} and {}".format(f1, f2))

Factors found : 315 * 3660068268593165 = 1152921504606846975
CPU times: user 36.7 s, sys: 2.79 s, total: 39.5 s
Wall time: 2.79 s
